In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#from matplotlib import rcParams
#import scipy.stats as st
from sklearn.model_selection import KFold, cross_val_score, train_test_split
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import RandomizedSearchCV # Model evaluation
from sklearn.metrics import median_absolute_error, mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Dropout, BatchNormalization
#from keras.layers.advanced_activations import PReLU
#from keras.layers.noise import GaussianDropout
#from keras.optimizers import Adam
#from keras.wrappers.scikit_learn import KerasRegressor
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import Imputer

In [ ]:
n_folds = 10

def rmse(model):
    kf = KFold(n_folds, shuffle=True, random_state=40).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
df_usa = pd.read_csv("D:/Columbia U/IEOR 4501 python/Project/kc_house_data.csv")

print("Price Min")
print(df_usa['price'].min())
print("Price Mean")
print(df_usa['price'].mean())
print("Price Median")
print(df_usa['price'].median())
print("Price Max")
print(df_usa['price'].max())
print("Price Std")
print(df_usa['price'].std())

df_usa = df_usa.merge(pd.get_dummies(df_usa.floors, drop_first=True, prefix='Floors'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.waterfront, drop_first=True, prefix='watFront'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.view, drop_first=True, prefix='View'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.condition, drop_first=True, prefix='Cond'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.grade, prefix='Grade'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.bedrooms, drop_first=True, prefix='Bedrooms'), left_index=True, right_index=True)

del df_usa['floors'],
del df_usa['waterfront']
del df_usa['view']
del df_usa['condition']
del df_usa['grade']
del df_usa['bedrooms']


df_usa.drop(['id', 'date'], axis=1, inplace=True)
X = df_usa.drop("price",axis=1).values
y = df_usa["price"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=3)

In [ ]:
### XGBoost ###

# for c in properties.columns:
#     if properties[c].dtype == 'object':
#         lbl = LabelEncoder()
#         lbl.fit(list(properties[c].values))
#         properties[c] = lbl.transform(list(properties[c].values))
thresh = 5 * 10**(-3)
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
#select features using threshold
selection = SelectFromModel(model, threshold=thresh, prefit=True)
select_X_train = selection.transform(X_train)
# test 
select_X_test = selection.transform(X_test)

xgb1 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

xgb1.fit(X_train, y_train)

y_hat = xgb.predict(X_test)

xgb.score(X_test,y_test)

n_folds = 10
kfold = KFold(n_splits=n_folds, random_state=42)
cv_results = cross_val_score(model, X_train, y_train, cv= kfold,scoring='r2', n_jobs=-1)



xgb_params = {
    'eta': 0.03,
    'max_depth': 10,
    'subsample': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'lambda': 0.8,   
    'alpha': 0.4,
    'silent': 1
}

dtrain = xgb.DMatrix(select_X_train, y_train)
dtest = xgb.DMatrix(select_X_test)

num_boost_rounds = 200
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred1 = model.predict(dtest)

median_absolute_error(xgb_pred1, y_test)
np.sqrt(MSE(xgb_pred1, y_test))

print(pd.DataFrame(xgb_pred1).head())

xgb_params = {
    'eta': 0.03,
    'max_depth': 10,
    'subsample': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(select_X_train, y_train)
dtest = xgb.DMatrix(select_X_test)

num_boost_rounds = 200

model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred2 = model.predict(dtest)

median_absolute_error(xgb_pred2, y_test)

np.sqrt(MSE(xgb_pred2, y_test))
##### COMBINE XGBOOST RESULTS
xgb_pred = XGB1_WEIGHT*xgb_pred1 + (1-XGB1_WEIGHT)*xgb_pred2



In [ ]:
####################
### RandomForest ###
####################


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 200, num = 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(select_X_train, y_train)

rf_random.best_params_

y_hat = rf_random.predict(select_X_test)

median_absolute_error(y_hat, y_test)

np.sqrt(MSE(y_hat, y_test))